In [27]:
from inspect import getsource
from ehrhart_quasi_polynomial import (
    PiecewiseEhrhartQuasiPolynomial as PEQP,
    _determine_lifts
)

In [2]:
A = Matrix([[-1, 0], [0, -1], [1, 2]])
p = PEQP(A)

In [3]:
lifts = p._cone_dicts[0]["lifts"]; lifts

{(0,): (0, 0, 0),
 (1,): (0, 0, 1),
 (2,): (0, 0, 2),
 (3,): (0, 0, 3),
 (4,): (0, 0, 4),
 (5,): (0, 0, 5)}

In [87]:
def _simplify_invariance_lattice(self, cone):
        invariants = tuple(lift for lift in list(cone["lifts"].values())[1:]
                           if self._check_periodicity(cone,
                                                      free_module_element(lift)))
        inv_basis = tuple(span(invariants).matrix().rows())
        cone["inv_basis"] = inv_basis
        cone["invariant"] = inv_basis + cone["scaled_rays"]
        remaining_keys = self._get_remaining_keys(cone["lifts"], inv_basis)

        cone["print"] = {tuple(cone["lifts"][key]): poly
                        for key, poly in cone["transformed"].items()
                        if cone["lifts"][key] in remaining_keys}

def _check_periodicity(self, cone, inv_cand):
        for rep, poly in cone["transformed"].items():
            translated = cone["lifts"][rep] + inv_cand
            trans_rep = cone["quotient"](translated)
            if poly != cone["transformed"][tuple(trans_rep)]:
                return False
        return True

def _get_remaining_keys(self, lifts, invariants):
    keys = list(lifts.values())
    keys_to_remove = []
    for inv in invariants:
        inv_norm = inv.norm()
    
        large_keys = [key for key in keys if key.norm() >= inv_norm]
    
        for l_key in large_keys:
            for key in keys:
                if l_key == key + inv:
                    keys_to_remove.append(l_key)

    remaining_keys = [key for key in keys if key not in keys_to_remove]
    return remaining_keys


PEQP._simplify_invariance_lattice = _simplify_invariance_lattice
PEQP._check_periodicity = _check_periodicity
PEQP._get_remaining_keys = _get_remaining_keys

In [88]:
p._transform_polynomials()

TypeError: generators must be lists of ring elements or free module elements!

In [51]:
def __str__(self):
        # key = "transformed"
        key = "print"

        cone_strings = []
        for cone in self._cone_dicts:
            rays = cone["rays"] + self._lin_vectors + self._minus_lin
            span = cone["invariant"]
            cone_str = f"On Cone{rays}\ninvariant on Lattice{span}:"

            rep_string = [str(rep) for rep in cone[key].keys()]
            poly_strings = [str(poly) for poly in cone[key].values()]
            max_poly_length = len(max(poly_strings, key=len)) + 2

            nice_polys = [f"\n    {poly}{' '*(max_poly_length-len(poly))} if [b] = {rep}"
                          for poly, rep in zip(poly_strings, rep_string)]

            cone_str += "".join(nice_polys)
            cone_strings.append(cone_str)

        return "\n\n".join(cone_strings)

PEQP.__str__ = __str__

In [66]:
print(p)

On Cone((1, 2, 1), (1, 0, -1), (0, 1, -2), (-1, 0, 1), (0, -1, 2))
invariant on Lattice((0, 0, 2), (1, 2, 1)):
    1/4*x0^2 + x0*x1 + x1^2 + 1/2*x0*x2 + x1*x2 + 1/4*x2^2 + x0 + 2*x1 + x2 + 1     if [b] = (0, 0, 0)
    1/4*x0^2 + x0*x1 + x1^2 + 1/2*x0*x2 + x1*x2 + 1/4*x2^2 + x0 + 2*x1 + x2 + 3/4   if [b] = (0, 0, 1)


In [67]:
cone = p._cone_dicts[0]